In [1]:
    !pip install datasets transformers evaluate imbalanced-learn umap-learn wandb GPUtil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=bc5eee260ac90fd5cd0f350d368676a1890d406dfd3fe2acee0eafc471c91765
  Stored in directory: /root/.cache/pip/wheels/b1/e7/99/2b32600270cf23194c9860f029d3d5db075f250bc39028c045
Successfully built GPUtil


# Tuning a Longformer model on a Custom Feature

In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [66]:
from datasets import load_dataset

health_fact = load_dataset('health_fact')

  0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
# Filter out instances with a -1 label
health_fact['train'] = health_fact['train'].filter(lambda x: x['label'] != -1)
health_fact['validation'] = health_fact['validation'].filter(lambda x: x['label'] != -1)
health_fact['test'] = health_fact['test'].filter(lambda x: x['label'] != -1)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [72]:
def label_int2str(row):
    if row == -1:
        return 'invalid'
    return health_fact["train"].features["label"].int2str(row)

In [9]:
from transformers import AutoTokenizer

# model_ckpt = 'distilbert-base-uncased'
model_ckpt = 'allenai/longformer-base-4096'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

### Constructing the Custom Feature

In [73]:
def new_column(example):
    example["summary"] = f"SUBJECTS: {example['subjects']} - CLAIM: {example['claim']} - EXPLANATION: {example['explanation']}"
    return example

health_fact = health_fact.map(new_column)
health_fact['train'][0]

  0%|          | 0/9804 [00:00<?, ?ex/s]

  0%|          | 0/1233 [00:00<?, ?ex/s]

  0%|          | 0/1214 [00:00<?, ?ex/s]

{'claim_id': '15661',
 'claim': '"The money the Clinton Foundation took from from foreign governments while Hillary Clinton was secretary of state ""is clearly illegal. … The Constitution says you can’t take this stuff."',
 'date_published': 'April 26, 2015',
 'explanation': '"Gingrich said the Clinton Foundation ""took money from from foreign governments while (Hillary Clinton) was secretary of state. It is clearly illegal. … The Constitution says you can’t take this stuff."" A clause in the Constitution does prohibit U.S. officials such as former Secretary of State Hillary Clinton from receiving gifts, or emoluments, from foreign governments. But the gifts in this case were donations from foreign governments that went to the Clinton Foundation, not Hillary Clinton. She was not part of the foundation her husband founded while she was secretary of state. Does that violate the Constitution? Some libertarian-minded constitutional law experts say it very well could. Others are skeptical. 

In [74]:
def tokenize(batch):
    return tokenizer(batch['summary'], truncation=True, padding=True)

In [75]:
health_fact_encoded = health_fact.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
print(health_fact_encoded["train"].column_names)

['claim_id', 'claim', 'date_published', 'explanation', 'fact_checkers', 'main_text', 'sources', 'label', 'subjects', 'summary', 'input_ids', 'attention_mask']


In [80]:
import torch
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
from transformers import AutoModelForSequenceClassification

num_labels = 4
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias'

In [18]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [16]:
from transformers import Trainer, TrainingArguments

batch_size = 4
logging_steps = len(health_fact_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-health-fact"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=3,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.0,
                                  warmup_steps=200,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  load_best_model_at_end = True,
                                  log_level="error",)


In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()

from transformers import Trainer
from transformers import EarlyStoppingCallback

class CustomEarlyStoppingCallback(EarlyStoppingCallback):
    def __init__(self, early_stopping_patience=3):
        super().__init__(early_stopping_patience=early_stopping_patience)

early_stopping_callback = CustomEarlyStoppingCallback(early_stopping_patience=1)
        
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=health_fact_encoded["train"],
                  eval_dataset=health_fact_encoded["validation"],
                  tokenizer=tokenizer,
                  callbacks=[early_stopping_callback],)
trainer.train();

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.739100,0.588144,0.759473,0.744247
2,0.584300,0.581178,0.787479,0.782440
3,0.515500,0.785792,0.785832,0.782677


## Results

We can run predictions over the validation and test sets for the best model to see how we did

In [20]:
trainer.save_model('longformer_on_summary')

In [21]:
preds_output = trainer.predict(health_fact_encoded["validation"])

In [22]:
preds_output.metrics

{'test_loss': 0.5811779499053955,
 'test_accuracy': 0.7874794069192751,
 'test_f1': 0.7824398745506814,
 'test_runtime': 82.9499,
 'test_samples_per_second': 14.635,
 'test_steps_per_second': 3.665}

In [34]:
test_preds_output = trainer.predict(health_fact_encoded["test"])

In [35]:
test_preds_output.metrics

{'test_loss': 0.7174000144004822,
 'test_accuracy': 0.7429034874290349,
 'test_f1': 0.7363373286634284,
 'test_runtime': 83.9016,
 'test_samples_per_second': 14.696,
 'test_steps_per_second': 3.683}

From the above we can see that we achieved an accuracy of ~0.742 and an f1 of ~0.736 on the held out test set

## HyperParameter Search on Distilbert

In [1]:
import wandb
wandb.init(project="pubhealth-hyperparameter-search")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [62]:
learning_rates = [2e-5, 3e-5, 5e-5]
batch_sizes = [8, 16]
weight_decays = [0.0, 0.01]
warmup_steps = [0, 100, 200]
idx = 1
for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
            for warmup_step in warmup_steps:
                print(idx, learning_rate, batch_size, weight_decay, warmup_step)
                idx += 1

1 2e-05 8 0.0 0
2 2e-05 8 0.0 100
3 2e-05 8 0.0 200
4 2e-05 8 0.01 0
5 2e-05 8 0.01 100
6 2e-05 8 0.01 200
7 2e-05 16 0.0 0
8 2e-05 16 0.0 100
9 2e-05 16 0.0 200
10 2e-05 16 0.01 0
11 2e-05 16 0.01 100
12 2e-05 16 0.01 200
13 3e-05 8 0.0 0
14 3e-05 8 0.0 100
15 3e-05 8 0.0 200
16 3e-05 8 0.01 0
17 3e-05 8 0.01 100
18 3e-05 8 0.01 200
19 3e-05 16 0.0 0
20 3e-05 16 0.0 100
21 3e-05 16 0.0 200
22 3e-05 16 0.01 0
23 3e-05 16 0.01 100
24 3e-05 16 0.01 200
25 5e-05 8 0.0 0
26 5e-05 8 0.0 100
27 5e-05 8 0.0 200
28 5e-05 8 0.01 0
29 5e-05 8 0.01 100
30 5e-05 8 0.01 200
31 5e-05 16 0.0 0
32 5e-05 16 0.0 100
33 5e-05 16 0.0 200
34 5e-05 16 0.01 0
35 5e-05 16 0.01 100
36 5e-05 16 0.01 200


In [42]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
num_labels=4
# Define the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Function to train and evaluate the model with given hyperparameters
def train_evaluate_model(learning_rate, batch_size, weight_decay, warmup_step):
    model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_step,
        logging_dir='./logs',
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        report_to="wandb",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
    )

    trainer = Trainer(
        model=model, 
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=health_fact_encoded["train"],
        eval_dataset=health_fact_encoded["validation"],
        tokenizer=tokenizer,
        callbacks=[CustomEarlyStoppingCallback(early_stopping_patience=1)],
    )

    # Train the model
    trainer.train()
    
    # would be good to run the below up front to check that this actually works ...

    # Evaluate the model on the validation set
    results = trainer.evaluate(health_fact_encoded["validation"])
    
    summary = {
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "weight_decay": weight_decay,
        "warmup_step": warmup_step,
        "accuracy": results["eval_accuracy"],
    }
    print(summary)
    # Log the hyperparameters and results in wandb
    wandb.log(summary)

# Perform hyperparameter search
learning_rates = [2e-5, 3e-5, 5e-5]
batch_sizes = [8, 16]
weight_decays = [0.0, 0.01]
warmup_steps = [0, 100, 200]

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for weight_decay in weight_decays:
            for warmup_step in warmup_steps:
                train_evaluate_model(learning_rate, batch_size, weight_decay, warmup_step)


# Close wandb
wandb.finish()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim. If main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9804
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4904
  Num

{'loss': 1.0252, 'learning_rate': 1.959216965742251e-05, 'epoch': 0.08}
{'loss': 0.8932, 'learning_rate': 1.9184339314845025e-05, 'epoch': 0.16}
{'loss': 0.801, 'learning_rate': 1.877650897226754e-05, 'epoch': 0.24}
{'loss': 0.7737, 'learning_rate': 1.8368678629690052e-05, 'epoch': 0.33}
{'loss': 0.7832, 'learning_rate': 1.7960848287112562e-05, 'epoch': 0.41}
{'loss': 0.7944, 'learning_rate': 1.7553017944535075e-05, 'epoch': 0.49}
{'loss': 0.7521, 'learning_rate': 1.714518760195759e-05, 'epoch': 0.57}
{'loss': 0.7588, 'learning_rate': 1.67373572593801e-05, 'epoch': 0.65}
{'loss': 0.7484, 'learning_rate': 1.6329526916802612e-05, 'epoch': 0.73}
{'loss': 0.7169, 'learning_rate': 1.5921696574225122e-05, 'epoch': 0.82}
{'loss': 0.6943, 'learning_rate': 1.5513866231647635e-05, 'epoch': 0.9}
{'loss': 0.6838, 'learning_rate': 1.5106035889070147e-05, 'epoch': 0.98}


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim. If main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1226
Configuration saved in ./results/checkpoint-1226/config.json


{'eval_loss': 0.7055772542953491, 'eval_accuracy': 0.71334431630972, 'eval_f1': 0.6903688677860969, 'eval_runtime': 11.1119, 'eval_samples_per_second': 109.252, 'eval_steps_per_second': 13.679, 'epoch': 1.0}


Model weights saved in ./results/checkpoint-1226/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1226/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1226/special_tokens_map.json


...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim. If main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-613
Configuration saved in ./results/checkpoint-613/config.json
Model weights saved in ./results/checkpoint-613/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-613/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-613/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: mai

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingfac

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.700100,0.671288,0.722405,0.705562
2,0.575300,0.677237,0.738056,0.721710
3,0.348400,0.752762,0.749588,0.745155
4,0.228200,0.919576,0.740527,0.738035


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim. If main_text, explanation, claim_id, sources, fact_checkers, date_published, subjects, claim are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1214
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-613
Configuration saved in ./results/checkpoint-613/config.json
Model weights saved in ./results/checkpoint-613/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-613/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-613/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: mai

accuracy,▄▆▇▆▆▇▁▅▅▇█▅▄▆▄▅▅▅▆▁▆▅▄▄▄▇▆▇▆▇▆▄█▅▅▇
batch_size,▁▁▁▁▁▁██████▁▁▁▁▁▁██████▁▁▁▁▁▁██████
eval/accuracy,▆▇▇▇▇▇▄▆▆▄▇█▆▆▆▅▆▃▆▆▆▄▇▆▆▆▆█▇█▆▁▇▆▆▃▄▆▅▇
eval/f1,▆▆▇▆▆▇▄▇▅▅██▇▇▇▆▅▃▇▇▇▅▆▇▇▆▆█▇▇▆▁█▇▇▄▅▇▆█
eval/loss,▁▁▁▁▁▁▁▁▁▁▁▂▁▇▄▃▂▁▃▆▂▁▁▃▂▁▂█▄▁▂▂▄▅▂▁▁▅▅▂
eval/runtime,▄▇▇▇▇▇▂▂▂▂▂▂▂▇▇█▇▇█▇▂▂▁▂▂▂█▇▇▇▆▆▇▂▂▃▂▁▁▂
eval/samples_per_second,▅▂▂▂▂▂▇▆▇▇▇▇▇▂▂▁▂▂▁▂▇▇█▇▇▇▁▂▂▂▂▃▂▇▇▆▇██▇
eval/steps_per_second,██████▁▁▁▁▁▁▁███████▁▁▁▁▁▁▇██████▁▁▁▁▁▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▃▃▃▃████████████
train/epoch,▃▂▆▅▄▂▁▆███▁▁▆▃▁█▅▂▆▃▇▁▅▄▄▄▃▇▄█▅▆▃▇▇██▁▆
train/global_step,▃▁▆▄▃▂▃▁▃▃▃▃▂▆▃▁▆▅▁▆▃▇▃▁▁▁▃▁█▅▁▆▄▁█▄▄▄▄▄


## Experimenting with Sweep Configuration

In [ ]:
# from transformers import AutoModelForSequenceClassification
# from transformers import Trainer, TrainingArguments
# num_labels=4
# # Define the model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# def main(verbose=False):
#     # Initialise run
#     run = wandb.init()
#     model = (AutoModelForSequenceClassification
#          .from_pretrained(model_ckpt, num_labels=num_labels)
#          .to(device))

#     training_args = TrainingArguments(
#         output_dir='./results',
#         num_train_epochs=4,
#         per_device_train_batch_size=wandb.config.batch_size,
#         per_device_eval_batch_size=wandb.config.batch_size,
#         learning_rate=wandb.config.learning_rate,
#         weight_decay=wandb.config.weight_decay,
#         dropout_rate=wandb.config.dropout_rate,
#         logging_dir='./logs',
#         logging_steps=100,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         save_total_limit=1,
#         report_to="wandb",
#         load_best_model_at_end=True,
#         metric_for_best_model="accuracy",
#         greater_is_better=True,
#         patience=1,  # Number of epochs with no improvement after which training will be stopped
#     )

#     trainer = Trainer(
#         model=model, 
#         args=training_args,
#         compute_metrics=compute_metrics,
#         train_dataset=health_fact_encoded["train"],
#         eval_dataset=health_fact_encoded["validation"],
#         tokenizer=tokenizer
#     )

#     # Train the model
#     trainer.train()

#     # Evaluate the model on the validation set
#     results = trainer.evaluate(health_fact_encoded["validation"])

#     # Log the hyperparameters and results in wandb
#     wandb.log({
#         "learning_rate": learning_rate,
#         "batch_size": batch_size,
#         "weight_decay": weight_decay,
#         "warmup_step": warmup_step,
#         "accuracy": results["eval_accuracy"],
#     })

In [ ]:
# sweep_configuration = {
#     'method': 'bayes',  # random, grid or bayes
#     'name': 'sweep-bayes',
#     'metric': {'goal': 'maximize', 'name': 'eval_accuracy'},
#     'parameters': 
#     {
#         'batch_size': {'values': [8, 16, 32]},
#         'learning_rate': {'max': 0.1, 'min': 0.0001},
#         'weight_decay': {'values': [0.0, 0.1, 0.2]},
#         'dropout_rate': {'max': 0.5, 'min': 0.1}
#      }
# }

In [ ]:
# sweep_id = wandb.sweep(sweep=sweep_configuration, entity='tansaku', project='pubhealth-hyperparameter-search')

In [ ]:
# # Maximum 'count' runs
# wandb.agent(sweep_id, function=main, count=30)